In [29]:
import pandas as pd

In [11]:
import json
import time
from time import sleep

# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.support.ui import WebDriverWait

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

# --크롬창을 숨기고 실행-- driver에 options를 추가해주면된다
# options = webdriver.ChromeOptions()
# options.add_argument('headless')

url = 'https://map.naver.com/v5/search'
# chrome_path='C:\\Users\\kim\\chromedriver-win64'  # 드라이버 경로
# # driver = webdriver.Chrome('./chromedriver',chrome_options=options) # 크롬창 숨기기
# options = webdriver.ChromeOptions()
# driver = webdriver.Chrome(executable_path=chrome_path, options=options)
driver.get(url)
key_word = '연희동 카페'  # 검색어

# css 찾을때 까지 10초대기
def time_wait(num, code):
    try:
        wait = WebDriverWait(driver, num).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, code)))
    except:
        print(code, '태그를 찾지 못하였습니다.')
        driver.quit()
    return wait

# frame 변경 메소드
def switch_frame(frame):
    driver.switch_to.default_content()  # frame 초기화
    driver.switch_to.frame(frame)  # frame 변경

# 페이지 다운
def page_down(num):
    body = driver.find_element(By.CSS_SELECTOR, 'body')
    body.click()
    for i in range(num):
        body.send_keys(Keys.PAGE_DOWN)

# css를 찾을때 까지 10초 대기
time_wait(10, 'div.input_box > input.input_search')

# (1) 검색창 찾기
search = driver.find_element(By.CSS_SELECTOR, 'div.input_box > input.input_search')
search.send_keys(key_word)  # 검색어 입력
search.send_keys(Keys.ENTER)  # 엔터버튼 누르기

sleep(1)

# (2) frame 변경
switch_frame('searchIframe')
page_down(40)
sleep(3)

# 카페 리스트
cafe_list = driver.find_elements(By.CSS_SELECTOR, 'li.UEzoS')
# 페이지 리스트
next_btn = driver.find_elements(By.CSS_SELECTOR, '.zRM9F> a')

# dictionary 생성
cafe_dict = {'카페정보': []}
# 시작시간
start = time.time()
print('[크롤링 시작...]')

# 크롤링 (페이지 리스트 만큼)
for btn in range(len(next_btn))[1:]:  # next_btn[0] = 이전 페이지 버튼 무시 -> [1]부터 시작
    cafe_list = driver.find_elements(By.CSS_SELECTOR, 'li.UEzoS')
    names = driver.find_elements(By.CSS_SELECTOR, '.TYaxT')  # (3) 장소명
    types = driver.find_elements(By.CSS_SELECTOR, '.KCMnt')  # (4) 장소 유형

    for data in range(len(cafe_list)):  # 카페 리스트 만큼
        print(data)

        sleep(1)
        try:
            # 지번, 도로명 초기화
            jibun_address = ''
            road_address = ''

            # (3) 카페명 가져오기
            cafe_name = names[data].text
            print(cafe_name)

            # (4) 유형
            cafe_type = types[data].text
            print(cafe_type)

            # dict에 데이터 집어넣기
            dict_temp = {
                'name': cafe_name,
                'cafe_type': cafe_type,
                # 'road_address': road_address,
                # 'jibun_address': jibun_address
            }

            cafe_dict['카페정보'].append(dict_temp)
            sleep(1)

        except Exception as e:
            print(e)
            print('ERROR!' * 3)

            dict_temp = {
                'name': cafe_name,
                'cafe_type': cafe_type

            }

            cafe_dict['카페정보'].append(dict_temp)
            sleep(1)

    # 다음 페이지 버튼 누를 수 없으면 종료
    if not next_btn[-1].is_enabled():
        break

    if names[-1]:  # 마지막 카페일 경우 다음버튼 클릭
        next_btn[-1].click()

        sleep(2)

    else:
        print('페이지 인식 못함')
        break

print('[데이터 수집 완료]\n소요 시간 :', time.time() - start)
driver.quit()  # 작업이 끝나면 창을 닫는다.

# 배열에 저장
data = [cafe_dict]


[크롤링 시작...]
0
Poke all day 포케&샐러드 연희점
다이어트,샐러드
1
그레인서울
브런치
2
스웨이커피스테이션
카페,디저트
3
프로토콜 연희점
카페,디저트
4
보연희
카페,디저트
5
푸어링아웃
카페,디저트
6
피터팬1978
베이커리
7
매뉴팩트커피
카페
8
아워온즈
카페,디저트
9
카페 마
카페,디저트
10
앤트러사이트 연희점
카페,디저트
11
에브리띵베이글
베이글
12
금옥당 연희점
카페,디저트
13
본지르르 연희
카페
14
더반 베를린 연희점
카페,디저트
15
폴앤폴리나
베이커리
16
연희 에스프레소 바
카페,디저트
17
카시코이
카페,디저트
18
보케
카페,디저트
19
시간이머무는홍차가게
홍차전문점
20
연희동 국화빵
베이커리
21
디폴트밸류
카페,디저트
22
팔레트샐러드 연희점
브런치
23
쿳사 연희
브런치
24
해피앤피스커피클럽
카페,디저트
25
연희동오븐
카페,디저트
26
비전스트롤 연희점
카페,디저트
27
lodge190
카페,디저트
28
알트
카페,디저트
29
로덴드론 연희 1호점
카페
30
컬러드 빈
카페
31
그로밋커피하우스
카페,디저트
32
다도레 티룸
차
33
카페 데스툴 연희점
카페,디저트
34
뚤리
카페,디저트
35
쿳사 연희
브런치
36
연희양과점 연희점
베이커리
37
베지스
브런치카페
38
피크닉베이크
카페,디저트
39
미도파
카페,디저트
0
혹스턴
카페
1
뉘블랑쉬
베이커리
2
알레스구떼
베이커리
3
제니스 까페
카페
4
감과당
카페,디저트
5
스타벅스 연희DT점
카페
6
그로어스
브런치
7
다크에디션커피
카페
8
카페 니어리
카페,디저트
9
포싵
케이크전문
10
티아레나
홍차전문점
11
프레스도넛 연희
카페,디저트
12
해피헤비드링커
카페,디저트
13
로우키 연희점
카페,디저트
14
파츄코 파라다이스
카페
15
콜드레시피
아이스크림
16
모와림
카페,디저트
17
객기
케이크전문
18
뺑드에떼
베이커리
19
하니앤손스
카페,디저트
20
시도 플레이스
카페,디저트
21
까사드선주
베이커리
22
스타벅스 연희동점
카페

In [27]:
data_list= data[0]['카페정보']

In [119]:
df = pd.DataFrame(data_list)
df

,name,cafe_type
0,Poke all day 포케&샐러드 연희점,"다이어트,샐러드"
1,그레인서울,브런치
2,스웨이커피스테이션,"카페,디저트"
3,프로토콜 연희점,"카페,디저트"
4,보연희,"카페,디저트"
...,...,...
235,카페드댕,"카페,디저트"
236,리히트,케이크전문
237,하우스오브바이닐 연남점,"카페,디저트"
238,커피하우스,"카페,디저트"


In [120]:
unique_name= df['name'].value_counts()

In [121]:
duplicate_names= unique_name[unique_name > 1].index

In [122]:
duplicate_names

Index(['그레인서울', '쿳사 연희', '팔레트샐러드 연희점'], dtype='object', name='name')

In [123]:
del_name=[]
for name in duplicate_names: 
    del_name.append(df[df['name'] == name].index[0])

In [124]:
del_name

[1, 23, 22]

In [125]:
df.drop(del_name, inplace=True)

In [126]:
df['name'].value_counts()

name
Poke all day 포케&샐러드 연희점    1
카페도비                       1
연희롱블랙                      1
이디야커피 서대문구청점               1
가비노                        1
                          ..
베리굳베이글 연희점                 1
벨르아티                       1
에스페란자 로스터즈 연희점             1
그레인서울                      1
오버딥                        1
Name: count, Length: 237, dtype: int64

In [127]:
df.to_csv('cafelist_연희동.csv',encoding='utf-8-sig')